# Serving a Session-based Recommendation model with Torch Backend

* following [this tutrial](https://github.com/NVIDIA-Merlin/Transformers4Rec/blob/main/examples/getting-started-session-based/03-serving-session-based-model-torch-backend.ipynb)

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf
import glob
import torch 

from transformers4rec import torch as tr
from merlin.io import Dataset

from merlin.core.dispatch import make_df  # noqa
from merlin.systems.dag import Ensemble  # noqa
from merlin.systems.dag.ops.pytorch import PredictPyTorch  # noqa

In [7]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
REGION = "us-central1"

# VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'
VERTEX_SA = 'jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")
print(f"REGION: {REGION}")
print(f"VERTEX_SA: {VERTEX_SA}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1
REGION: us-central1
VERTEX_SA: jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com


In [8]:
# INPUT_DATA_DIR = os.environ.get("INPUT_DATA_DIR", "/workspace/data")
# OUTPUT_DIR = os.environ.get("OUTPUT_DIR", f"{INPUT_DATA_DIR}/sessions_by_day")
# model_path= os.environ.get("model_path", f"{INPUT_DATA_DIR}/saved_model")

REPO_WORKSPACE = 'workspace'
DATA_DIR = 'data'
INPUT_DATA_DIR=f'{REPO_WORKSPACE}/{DATA_DIR}'
TRANSFORMED_WORKFLOW=f'{INPUT_DATA_DIR}/processed_nvt'
OUTPUT_DIR=f'{INPUT_DATA_DIR}/sessions_by_day'
MODEL_PATH = f'{INPUT_DATA_DIR}/saved_model'
# MODEL_PATH = './torch_ensemble/models'

print(f"INPUT_DATA_DIR: {INPUT_DATA_DIR}")
# print(f"TRANSFORMED_WORKFLOW: {TRANSFORMED_WORKFLOW}")
print(f"OUTPUT_DIR: {OUTPUT_DIR}")
print(f"MODEL_PATH: {MODEL_PATH}")

INPUT_DATA_DIR: workspace/data
OUTPUT_DIR: workspace/data/sessions_by_day
MODEL_PATH: workspace/data/saved_model


## Set the schema object

In [9]:
from merlin_standard_lib import Schema
# SCHEMA_PATH = os.environ.get("INPUT_SCHEMA_PATH", "/workspace/data/processed_nvt/schema.pbtxt")

SCHEMA_PATH = f'{TRANSFORMED_WORKFLOW}/schema.pbtxt'
schema = Schema().from_proto_text(SCHEMA_PATH)

In [10]:
import cloudpickle
loaded_model = cloudpickle.load(
                open(os.path.join(MODEL_PATH, "t4rec_model_class.pkl"), "rb")
            )

In [11]:
model = loaded_model.cuda()
model.eval()

Model(
  (heads): ModuleList(
    (0): Head(
      (body): SequentialBlock(
        (0): TabularSequenceFeatures(
          (to_merge): ModuleDict(
            (continuous_module): SequentialBlock(
              (0): ContinuousFeatures(
                (filter_features): FilterFeatures()
                (_aggregation): ConcatFeatures()
              )
              (1): SequentialBlock(
                (0): DenseBlock(
                  (0): Linear(in_features=2, out_features=64, bias=True)
                  (1): ReLU(inplace=True)
                )
              )
              (2): AsTabular()
            )
            (categorical_module): SequenceEmbeddingFeatures(
              (filter_features): FilterFeatures()
              (embedding_tables): ModuleDict(
                (item_id-list): Embedding(491, 64, padding_idx=0)
                (category-list): Embedding(147, 64, padding_idx=0)
              )
            )
          )
          (_aggregation): ConcatFeatures()
        

## Trace the model


In [12]:
# train_paths = os.path.join(OUTPUT_DIR, f"{1}/train.parquet")
TRAIN_PATHS=f'{OUTPUT_DIR}/1/train.parquet'
dataset = Dataset(TRAIN_PATHS)

In [13]:
TRAIN_PATHS

'workspace/data/sessions_by_day/1/train.parquet'

In [14]:
sparse_max = {'age_days-list': 20,
 'weekday_sin-list': 20,
 'item_id-list': 20,
 'category-list': 20}

from transformers4rec.torch.utils.data_utils import MerlinDataLoader

def generate_dataloader(schema, dataset, batch_size=128, seq_length=20):
    loader = MerlinDataLoader.from_schema(
            schema,
            dataset,
            batch_size=batch_size,
            max_sequence_length=seq_length,
            shuffle=False,
            sparse_as_dense=True,
            sparse_max=sparse_max
        )
    return loader

In [15]:
loader = generate_dataloader(schema, dataset)
train_dict = next(iter(loader))

In [16]:
train_dict[0]['item_id-list']

tensor([[ 49,  36, 130,  ...,   0,   0,   0],
        [ 34,  16,  34,  ...,   0,   0,   0],
        [  3,   8,  14,  ...,   0,   0,   0],
        ...,
        [  4,  17,  95,  ...,   0,   0,   0],
        [ 23,   2,  23,  ...,   0,   0,   0],
        [174, 103,  25,  ...,   0,   0,   0]], device='cuda:0')

In [17]:
train_dict[0]

{'age_days-list': tensor([[0.4849, 0.8648, 0.1370,  ..., 0.0000, 0.0000, 0.0000],
         [0.9514, 0.9971, 0.5264,  ..., 0.0000, 0.0000, 0.0000],
         [0.5986, 0.7829, 0.8959,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.8389, 0.7179, 0.2957,  ..., 0.0000, 0.0000, 0.0000],
         [0.7645, 0.2106, 0.6182,  ..., 0.0000, 0.0000, 0.0000],
         [0.5982, 0.4947, 0.6335,  ..., 0.0000, 0.0000, 0.0000]],
        device='cuda:0'),
 'weekday_sin-list': tensor([[0.0140, 0.3720, 0.5400,  ..., 0.0000, 0.0000, 0.0000],
         [0.5489, 0.3369, 0.8028,  ..., 0.0000, 0.0000, 0.0000],
         [0.2827, 0.8099, 0.8618,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0313, 0.6036, 0.2137,  ..., 0.0000, 0.0000, 0.0000],
         [0.3610, 0.7434, 0.2133,  ..., 0.0000, 0.0000, 0.0000],
         [0.8821, 0.7026, 0.4364,  ..., 0.0000, 0.0000, 0.0000]],
        device='cuda:0'),
 'item_id-list': tensor([[ 49,  36, 130,  ...,   0,   0,   0],
         [ 34,  16,  34,  ...,   0,   

In [18]:
traced_model = torch.jit.trace(model, train_dict[0], strict=True)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [157]:
# traced_model

### Generate model input and output schemas to feed in the PredictPyTorch operator below.

In [20]:
input_schema = model.input_schema
output_schema = model.output_schema

In [21]:
input_schema

,name,tags,dtype,is_list,is_ragged,properties.int_domain.min,properties.int_domain.max
0,age_days-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,False,0,0
1,weekday_sin-list,"(Tags.LIST, Tags.CONTINUOUS)",float32,True,False,0,0
2,item_id-list,"(Tags.ITEM_ID, Tags.ITEM, Tags.CATEGORICAL, Ta...",int64,True,False,0,490
3,category-list,"(Tags.CATEGORICAL, Tags.LIST)",int64,True,False,0,146


In [22]:
INPUT_DATA_DIR

'workspace/data'

In [118]:
import shutil
# ens_model_path = os.environ.get("ens_model_path", f"{INPUT_DATA_DIR}/models")

ens_model_path = f'workspace/data/models'

# Make sure we have a clean stats space for Dask
if os.path.isdir(ens_model_path):
    shutil.rmtree(ens_model_path)
os.mkdir(ens_model_path)

### We use `PredictPyTorch` operator that takes a pytorch model and packages it correctly for tritonserver to run on the PyTorch backend.

In [26]:
torch_op = input_schema.column_names >> PredictPyTorch(
    traced_model, input_schema, output_schema
)

In [27]:
ensemble = Ensemble(torch_op, input_schema)
ens_config, node_configs = ensemble.export(ens_model_path)

# Starting Triton Server

In [124]:
ens_model_path

'workspace/data/models'

In [120]:
# tritonserver --model-repository=/workspace/data/models

In [138]:
import tritonclient.http as client

# Create a triton client
try:
    triton_client = client.InferenceServerClient(url="localhost:8000", verbose=True)
    print("client created.")
except Exception as e:
    print("channel creation failed: " + str(e))

client created.


In [139]:
# ensure triton is in a good state
triton_client.is_server_live()
triton_client.get_model_repository_index()

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>
POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '121'}>
bytearray(b'[{"name":"0_predictpytorchtriton","version":"1","state":"READY"},{"name":"ensemble_model","version":"1","state":"READY"}]')


[{'name': '0_predictpytorchtriton', 'version': '1', 'state': 'READY'},
 {'name': 'ensemble_model', 'version': '1', 'state': 'READY'}]

In [140]:
! curl -s -o /dev/null -w "%{http_code}" http://localhost:8000/v2/health/ready

200

In [141]:
eval_batch_size = 32
# eval_paths = os.path.join(OUTPUT_DIR, f"{1}/valid.parquet")
eval_paths=f'{OUTPUT_DIR}/1/valid.parquet'

eval_dataset = Dataset(eval_paths, shuffle=False)
eval_loader = generate_dataloader(schema, eval_dataset, batch_size=eval_batch_size)
test_dict = next(iter(eval_loader))

In [142]:
test_dict[0]['item_id-list'][0]

tensor([67,  7, 44,  4,  4, 83, 96,  9,  7, 99, 12, 16,  0,  0,  0,  0,  0,  0,
         0,  0], device='cuda:0')

In [143]:
# test_dict

In [144]:
df_cols = {}
for name, tensor in test_dict[0].items():
    if name in input_schema.column_names:
        df_cols[name] = tensor.cpu().numpy()
        if len(tensor.shape) > 1:
            df_cols[name] = list(df_cols[name])
            
df = make_df(df_cols)
print(df.shape)
df.head()

(32, 4)


,age_days-list,weekday_sin-list,item_id-list,category-list
0,"[0.5499501, 0.05859168, 0.6143423, 0.6127391, ...","[0.14183702, 0.033720933, 0.33820504, 0.261525...","[67, 7, 44, 4, 4, 83, 96, 9, 7, 99, 12, 16, 0,...","[15, 2, 9, 1, 1, 18, 20, 2, 2, 22, 2, 4, 0, 0,..."
1,"[0.054626163, 0.9909682, 0.06476167, 0.1239693...","[0.7294668, 0.19679245, 0.9772184, 0.018398901...","[7, 8, 77, 3, 11, 15, 8, 29, 67, 36, 0, 0, 0, ...","[2, 3, 16, 1, 2, 4, 3, 6, 15, 8, 0, 0, 0, 0, 0..."
2,"[0.932309, 0.80796856, 0.90713656, 0.15237969,...","[0.30309433, 0.32139105, 0.9710796, 0.2957704,...","[3, 17, 3, 11, 93, 152, 6, 54, 70, 54, 0, 0, 0...","[1, 4, 1, 2, 19, 32, 1, 11, 14, 11, 0, 0, 0, 0..."
3,"[0.54140306, 0.27389532, 0.71741486, 0.3560422...","[0.97362846, 0.36598137, 0.4796154, 0.5809341,...","[17, 98, 2, 32, 18, 22, 30, 2, 121, 22, 0, 0, ...","[4, 22, 1, 6, 4, 5, 6, 1, 25, 5, 0, 0, 0, 0, 0..."
4,"[0.12162325, 0.49388522, 0.31290212, 0.0548800...","[0.27499816, 0.40137193, 0.7278469, 0.2063758,...","[108, 1, 26, 121, 62, 35, 24, 15, 8, 0, 0, 0, ...","[23, 1, 7, 25, 13, 8, 5, 4, 3, 0, 0, 0, 0, 0, ..."


In [147]:
from merlin.systems.triton.utils import send_triton_request
response = send_triton_request(input_schema, df[input_schema.column_names], output_schema.column_names)
print(response)

{'next-item': array([[ -9.414455 ,  -3.3946676,  -3.5153584, ...,  -9.280288 ,
         -9.485153 , -10.177757 ],
       [ -9.41456  ,  -3.3946674,  -3.5153627, ...,  -9.280386 ,
         -9.48469  , -10.178482 ],
       [ -9.41453  ,  -3.394696 ,  -3.5153177, ...,  -9.280301 ,
         -9.485351 , -10.177533 ],
       ...,
       [ -9.414583 ,  -3.3947313,  -3.5153112, ...,  -9.280373 ,
         -9.485293 , -10.177692 ],
       [ -9.414576 ,  -3.39471  ,  -3.515358 , ...,  -9.280357 ,
         -9.485056 , -10.177971 ],
       [ -9.414605 ,  -3.394704 ,  -3.5153375, ...,  -9.280365 ,
         -9.485054 , -10.177982 ]], dtype=float32)}


In [156]:
response['next-item'].shape

(32, 491)

In [146]:
response['next-item'].shape

(32, 491)

In [ ]:
# REPO_WORKSPACE = 'workspace'

# DATA_DIR = 'data'
# INPUT_DATA_DIR=f'{REPO_WORKSPACE}/{DATA_DIR}'
# TRANSFORMED_WORKFLOW=f'{INPUT_DATA_DIR}/processed_nvt'
# OUTPUT_WORKFLOW_DIR=f'{INPUT_DATA_DIR}/workflow_etl'
# OUTPUT_DIR=f'{INPUT_DATA_DIR}/sessions_by_day'
# TRAIN_PATHS=f'{OUTPUT_DIR}/1' #/train.parquet'
# MODEL_PATH = f'{INPUT_DATA_DIR}/saved_model'

# print(f"INPUT_DATA_DIR: {INPUT_DATA_DIR}")
# print(f"TRANSFORMED_WORKFLOW: {TRANSFORMED_WORKFLOW}")
# print(f"OUTPUT_WORKFLOW_DIR: {OUTPUT_WORKFLOW_DIR}")
# print(f"OUTPUT_DIR: {OUTPUT_DIR}")
# print(f"TRAIN_PATHS: {TRAIN_PATHS}")
# print(f"MODEL_PATH: {MODEL_PATH}")

# Testing Triton locally with Vertex `LocalModel()`

### WIP - not stable

**TODO**

In [41]:
LOCAL_MODEL_ARTIFACTS_DIR = "local_model_artifacts"

In [137]:
!pwd

/home/jupyter/t4rec-nvidia-docs


In [42]:
MODEL_NAME = "local_torch"
LOCAL_MODEL_DIRECTORY = f"{LOCAL_MODEL_ARTIFACTS_DIR}/{MODEL_NAME}"
LOCAL_MODEL_VERSION_1 = f"{LOCAL_MODEL_DIRECTORY}/1"

!mkdir $LOCAL_MODEL_ARTIFACTS_DIR
!mkdir $LOCAL_MODEL_DIRECTORY
!mkdir $LOCAL_MODEL_VERSION_1

In [43]:
LOCAL_MODEL_FILE = f"{LOCAL_MODEL_VERSION_1}/model.pt"

# Use torch.jit.trace to generate a torch.jit.ScriptModule via tracing.
# traced_script_module = torch.jit.trace(model, example)

# Save the TorchScript model
traced_model.save(LOCAL_MODEL_FILE)

In [48]:
loaded_torch = torch.jit.load(LOCAL_MODEL_FILE)

def predict_class(instances):
    instances = torch.Tensor(instances)
    output = loaded_torch(instances)
    _, predicted = torch.max(output, 1)
    return predicted


# predicted = predict_class(test_dict[0])
# print([CLASS_VOCAB[class_index] for class_index in predicted])

In [49]:
predicted = loaded_torch(test_dict[0])

predicted

tensor([[ -9.4145,  -3.3947,  -3.5154,  ...,  -9.2803,  -9.4852, -10.1778],
        [ -9.4146,  -3.3947,  -3.5154,  ...,  -9.2804,  -9.4847, -10.1785],
        [ -9.4145,  -3.3947,  -3.5153,  ...,  -9.2803,  -9.4854, -10.1775],
        ...,
        [ -9.4146,  -3.3947,  -3.5153,  ...,  -9.2804,  -9.4853, -10.1777],
        [ -9.4146,  -3.3947,  -3.5154,  ...,  -9.2804,  -9.4851, -10.1780],
        [ -9.4146,  -3.3947,  -3.5153,  ...,  -9.2804,  -9.4851, -10.1780]],
       device='cuda:0', grad_fn=<LogSoftmaxBackward0>)

In [1]:
# test_dict

In [50]:
%%writefile $LOCAL_MODEL_DIRECTORY/config.pbtxt
name: "local_t4r_torch"
platform: "pytorch_libtorch"
input {
  name: "age_days-list"
  data_type: TYPE_FP32
  dims: -1
  dims: -1
}
input {
  name: "weekday_sin-list"
  data_type: TYPE_FP32
  dims: -1
  dims: -1
}
input {
  name: "item_id-list"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
input {
  name: "category-list"
  data_type: TYPE_INT64
  dims: -1
  dims: -1
}
output {
  name: "next-item"
  data_type: TYPE_FP32
  dims: -1
}
parameters {
  key: "INFERENCE_MODE"
  value {
    string_value: "true"
  }
}
backend: "pytorch"


Writing local_model_artifacts/local_torch/config.pbtxt


In [56]:
BUCKET_NAME='merlin-transformers4rec-jvt01'
BUCKET_URI=f'gs://{BUCKET_NAME}'

!gcloud storage cp -r $LOCAL_MODEL_ARTIFACTS_DIR/* $BUCKET_URI/local_model_dir/

Copying file://local_model_artifacts/local_torch/config.pbtxt to gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/config.pbtxt
Copying file://local_model_artifacts/local_torch/1/model.pt to gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/1/model.pt
  Completed files 2/2 | 795.2kiB/795.2kiB                                      

Average throughput: 75.9MiB/s


In [57]:
!gcloud storage ls $BUCKET_URI/local_model_dir/

gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch/


In [58]:
TRITON_VERSION = "23.01"
MODEL_NAME='local_t4r_torch'

In [110]:
from google.cloud.aiplatform.prediction import LocalModel

local_model = LocalModel(
    serving_container_image_uri='gcr.io/hybrid-vertex/triton-ensemble_model-v02', #f"nvcr.io/nvidia/tritonserver:{TRITON_VERSION}-py3",
    # serving_container_predict_route=f"/v1/models/{MODEL_NAME}/infer",
    # serving_container_health_route=f"/v1/models/{MODEL_NAME}",
    serving_container_ports=[8000],
    serving_container_args=[
        "tritonserver", 
        "--model-repository=./workspace/data/models/", 
        "--backend-config=python,shm-default-byte-size=4194304",
        "--vertex-ai-default-model=0_predictpytorchtriton",
        "--strict-model-config=true",
    ],
    # serving_container_environment_variables=
)
# tritonserver --model-repository=./models/  --backend-config=python,shm-default-byte-size=4194304 

# ./workspace/data/models/
# "--model-repository=$(AIP_STORAGE_URI)",

In [111]:
local_model.get_serving_container_spec()

image_uri: "gcr.io/hybrid-vertex/triton-ensemble_model-v02"
args: "tritonserver"
args: "--model-repository=./workspace/data/models/"
args: "--backend-config=python,shm-default-byte-size=4194304"
args: "--vertex-ai-default-model=0_predictpytorchtriton"
args: "--strict-model-config=true"
ports {
  container_port: 8000
}

GET /v2/health/live, headers None
<HTTPSocketPoolResponse status=200 headers={'content-length': '0', 'content-type': 'text/plain'}>
POST /v2/repository/index, headers None

<HTTPSocketPoolResponse status=200 headers={'content-type': 'application/json', 'content-length': '121'}>
bytearray(b'[{"name":"0_predictpytorchtriton","version":"1","state":"READY"},{"name":"ensemble_model","version":"1","state":"READY"}]')
[{'name': '0_predictpytorchtriton', 'version': '1', 'state': 'READY'},
 {'name': 'ensemble_model', 'version': '1', 'state': 'READY'}]

### GCP credentials

In [106]:
from google.oauth2 import service_account

CREDENTIALS_FILE = "credentials.json"
# t4rec-nvidia-docs/credentials.json
credentials = service_account.Credentials.from_service_account_file('credentials.json')

LOCAL_GCS_URI='gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch'

In [107]:
LOCAL_MODEL_ARTIFACTS_DIR

'local_model_artifacts'

In [108]:
from google.cloud import aiplatform as vertex_ai

vertex_ai.init(
    project=PROJECT_ID, 
    location=REGION,
    # credentials=credentials
)

In [113]:
# local_endpoint = local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_MODEL_DIRECTORY}",
#     # credential_path=CREDENTIALS_FILE,
# )
# local_endpoint.serve()

In [ ]:
health_check_response = local_endpoint.run_health_check()
print(health_check_response, health_check_response.content)

In [ ]:
predict_response = local_endpoint.predict(
    request='{"instances": [[1, 2, 3, 4]]}',
    headers={"header-key": "header-value"},
)
print(predict_response, predict_response.content)

In [ ]:
local_endpoint.print_container_logs()

In [ ]:
# local_endpoint.stop()

In [114]:
# # pas local paths
# with local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_MODEL_DIRECTORY}",
#     credential_path=CREDENTIALS_FILE,
# ) as local_endpoint:
#     health_check_response = local_endpoint.run_health_check()
#     print(health_check_response, health_check_response.content)
#     predict_response = local_endpoint.predict(
#         request_file=INPUT_FILE,
#         headers={"Content-Type": "application/json"},
#     )

    # health_check_response = local_endpoint.run_health_check()



# Pass GCP paths
# with local_model.deploy_to_local_endpoint(
#     artifact_uri=f"{LOCAL_GCS_URI}",
#     credential_path=CREDENTIALS_FILE,
# ) as local_endpoint:
#     predict_response = local_endpoint.predict(
#         request_file=INPUT_FILE,
#         headers={"Content-Type": "application/json"},
#     )

#     health_check_response = local_endpoint.run_health_check()

# >> DockerException: Error while fetching server API version: ('Connection aborted.', FileNotFoundError(2, 'No such file or directory'))

In [115]:
# LOCAL_GCS_URI='gs://merlin-transformers4rec-jvt01/local_model_dir/local_torch'

In [116]:
# INPUT_FILE